In [1]:
# try:
import numpy
import pandas as pd
import pickle as pk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
from nltk.stem.snowball import SnowballStemmer
import nltk
stemmer = SnowballStemmer("english")
import numpy
import pandas as pd
import pickle as pk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
from nltk.stem.snowball import SnowballStemmer
import nltk
stemmer = SnowballStemmer("english")
print('Done!')

Done!


In [2]:
#Input abstract
ARTICLE_READ=""" We discussed quantum deformations of D=4 Lorentz and Poincare algebras. In
the case of Poincare algebra it is shown that almost all classical r-matrices
of S. Zakrzewski classification correspond to twisted deformations of Abelian
and Jordanian types. A part of twists corresponding to the r-matrices of
Zakrzewski classification are given in explicit form.
"""

In [3]:
RES_ARTICLES="D:\Mini_project\Research_article_recommendation_system\data\project_data.csv"
# ARTICLE_READ=[1,10]
NUM_RECOMMENDED_ARTICLES=5

In [4]:
res_articles = pd.read_csv(RES_ARTICLES,encoding='utf-8',index_col=0)
#drop all the unnamed columns
res_articles.drop(res_articles.columns[res_articles.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
res_articles.head()

,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,abstract,...,authors_parsed/17/3,authors_parsed/18/3,authors_parsed/19/3,authors_parsed/20/3,authors_parsed/21/3,authors_parsed/22/3,authors_parsed/23/3,authors_parsed/24/3,authors_parsed/25/3,authors_parsed/26/3
id,,,,,,,,,,,,,,,,,,,,,
0704.0001,Pavel Nadolsky,"C. Bal\'azs, E. L. Berger, P. M. Nadolsky, C.-...",Calculation of prompt diphoton production cros...,"37 pages, 15 figures; published version","Phys.Rev.D76:013009,2007",10.1103/PhysRevD.76.013009,ANL-HEP-PR-07-12,hep-ph,NaN,A fully differential calculation in perturba...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0704.0002,Louis Theran,Ileana Streinu and Louis Theran,Sparsity-certifying Graph Decompositions,To appear in Graphs and Combinatorics,NaN,NaN,NaN,math.CO cs.CG,http://arxiv.org/licenses/nonexclusive-distrib...,"We describe a new algorithm, the $(k,\ell)$-...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0704.0003,Hongjun Pan,Hongjun Pan,The evolution of the Earth-Moon system based o...,"23 pages, 3 figures",NaN,NaN,NaN,physics.gen-ph,NaN,The evolution of Earth-Moon system is descri...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0704.0004,David Callan,David Callan,A determinant of Stirling cycle numbers counts...,11 pages,NaN,NaN,NaN,math.CO,NaN,We show that a determinant of Stirling cycle...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0704.0005,Alberto Torchinsky,Wael Abu-Shammala and Alberto Torchinsky,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...,NaN,"Illinois J. Math. 52 (2008) no.2, 681-689",NaN,NaN,math.CA math.FA,NaN,In this paper we show how to compute the $\L...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
res_ids = res_articles.index.tolist()
res_ids

['0704.0001',
 '0704.0002',
 '0704.0003',
 '0704.0004',
 '0704.0005',
 '0704.0006',
 '0704.0007',
 '0704.0008',
 '0704.0009',
 '0704.0010',
 '0704.0011',
 '0704.0012',
 '0704.0013',
 '0704.0014',
 '0704.0015',
 '0704.0016',
 '0704.0017',
 '0704.0018',
 '0704.0019',
 '0704.0020',
 '0704.0021',
 '0704.0022',
 '0704.0023',
 '0704.0024',
 '0704.0025',
 '0704.0026',
 '0704.0027',
 '0704.0028',
 '0704.0029',
 '0704.0030',
 '0704.0031',
 '0704.0032',
 '0704.0033',
 '0704.0034',
 '0704.0035',
 '0704.0036',
 '0704.0037',
 '0704.0038',
 '0704.0039',
 '0704.0040',
 '0704.0041',
 '0704.0042',
 '0704.0043',
 '0704.0044',
 '0704.0045',
 '0704.0046',
 '0704.0047',
 '0704.0048',
 '0704.0049',
 '0704.0050',
 '0704.0051',
 '0704.0052',
 '0704.0053',
 '0704.0054',
 '0704.0055',
 '0704.0056',
 '0704.0057',
 '0704.0058',
 '0704.0059',
 '0704.0060',
 '0704.0061',
 '0704.0062',
 '0704.0063',
 '0704.0064',
 '0704.0065',
 '0704.0066',
 '0704.0067',
 '0704.0068',
 '0704.0069',
 '0704.0070',
 '0704.0071',
 '0704

In [6]:
res_articles = res_articles[['title','abstract']].dropna()
#articles is a list of all articles
articles = res_articles['abstract'].tolist()
articles[0] #an uncleaned article

'  A fully differential calculation in perturbative quantum chromodynamics is\r\npresented for the production of massive photon pairs at hadron colliders. All\r\nnext-to-leading order perturbative contributions from quark-antiquark,\r\ngluon-(anti)quark, and gluon-gluon subprocesses are included, as well as\r\nall-orders resummation of initial-state gluon radiation valid at\r\nnext-to-next-to-leading logarithmic accuracy. The region of phase space is\r\nspecified in which the calculation is most reliable. Good agreement is\r\ndemonstrated with data from the Fermilab Tevatron, and predictions are made for\r\nmore detailed tests with CDF and DO data. Predictions are shown for\r\ndistributions of diphoton pairs produced at the energy of the Large Hadron\r\nCollider (LHC). Distributions of the diphoton pairs from the decay of a Higgs\r\nboson are contrasted with those produced from QCD processes at the LHC, showing\r\nthat enhanced sensitivity to the signal can be obtained with judicious\r

In [7]:
res_ids

['0704.0001',
 '0704.0002',
 '0704.0003',
 '0704.0004',
 '0704.0005',
 '0704.0006',
 '0704.0007',
 '0704.0008',
 '0704.0009',
 '0704.0010',
 '0704.0011',
 '0704.0012',
 '0704.0013',
 '0704.0014',
 '0704.0015',
 '0704.0016',
 '0704.0017',
 '0704.0018',
 '0704.0019',
 '0704.0020',
 '0704.0021',
 '0704.0022',
 '0704.0023',
 '0704.0024',
 '0704.0025',
 '0704.0026',
 '0704.0027',
 '0704.0028',
 '0704.0029',
 '0704.0030',
 '0704.0031',
 '0704.0032',
 '0704.0033',
 '0704.0034',
 '0704.0035',
 '0704.0036',
 '0704.0037',
 '0704.0038',
 '0704.0039',
 '0704.0040',
 '0704.0041',
 '0704.0042',
 '0704.0043',
 '0704.0044',
 '0704.0045',
 '0704.0046',
 '0704.0047',
 '0704.0048',
 '0704.0049',
 '0704.0050',
 '0704.0051',
 '0704.0052',
 '0704.0053',
 '0704.0054',
 '0704.0055',
 '0704.0056',
 '0704.0057',
 '0704.0058',
 '0704.0059',
 '0704.0060',
 '0704.0061',
 '0704.0062',
 '0704.0063',
 '0704.0064',
 '0704.0065',
 '0704.0066',
 '0704.0067',
 '0704.0068',
 '0704.0069',
 '0704.0070',
 '0704.0071',
 '0704

In [8]:
def clean_text(document):
    document = re.sub('[^\w_\s-]', ' ',document)       #remove punctuation marks and other symbols
    tokens = nltk.word_tokenize(document)              #Tokenize sentences
    cleaned_article = ' '.join([stemmer.stem(item) for item in tokens])    #Stemming each token
    return cleaned_article

In [9]:
cleaned_articles = list(map(clean_text, articles))
cleaned_articles[0]  #a cleaned, tokenized and stemmed article

'a fulli differenti calcul in perturb quantum chromodynam is present for the product of massiv photon pair at hadron collid all next-to-lead order perturb contribut from quark-antiquark gluon- anti quark and gluon-gluon subprocess are includ as well as all-ord resumm of initial-st gluon radiat valid at next-to-next-to-lead logarithm accuraci the region of phase space is specifi in which the calcul is most reliabl good agreement is demonstr with data from the fermilab tevatron and predict are made for more detail test with cdf and do data predict are shown for distribut of diphoton pair produc at the energi of the larg hadron collid lhc distribut of the diphoton pair from the decay of a higg boson are contrast with those produc from qcd process at the lhc show that enhanc sensit to the signal can be obtain with judici select of event'

In [10]:
user_articles = clean_text(ARTICLE_READ)
user_articles

'we discuss quantum deform of d 4 lorentz and poincar algebra in the case of poincar algebra it is shown that almost all classic r-matric of s zakrzewski classif correspond to twist deform of abelian and jordanian type a part of twist correspond to the r-matric of zakrzewski classif are given in explicit form'

In [11]:
#Generate tfidf matrix model for entire corpus
tfidf_matrix = TfidfVectorizer(stop_words='english', min_df=2) 
# min_df : When building the vocabulary ignore terms that have a document frequency strictly lower than the given threshold
article_tfidf_matrix = tfidf_matrix.fit_transform(cleaned_articles)
article_tfidf_matrix #tfidf vector of an article

<1918x4965 sparse matrix of type '<class 'numpy.float64'>'
	with 96335 stored elements in Compressed Sparse Row format>

In [12]:
#Generate tfidf matrix model for read articles
user_article_tfidf_vector = tfidf_matrix.transform([user_articles])
user_article_tfidf_vector

<1x4965 sparse matrix of type '<class 'numpy.float64'>'
	with 18 stored elements in Compressed Sparse Row format>

In [13]:
user_article_tfidf_vector.toarray()

array([[0., 0., 0., ..., 0., 0., 0.]])

In [14]:
articles_similarity_score=cosine_similarity(article_tfidf_matrix, user_article_tfidf_vector)

In [15]:
articles_similarity_score

array([[0.014237  ],
       [0.        ],
       [0.        ],
       ...,
       [0.01020967],
       [0.        ],
       [0.        ]])

In [16]:
recommended_articles_id = articles_similarity_score.flatten().argsort()[::-1]
recommended_articles_id

array([ 911,  630,  814, ..., 1055, 1051,  958], dtype=int64)

In [17]:
#Remove read articles from recommendations
temp_final_id = [article_id for article_id in recommended_articles_id][:NUM_RECOMMENDED_ARTICLES]
final_recommended_articles_id = [res_ids[i] for i in temp_final_id]
final_recommended_articles_id

['0704.0921', '0704.0640', '0704.0824', 'chao-dyn/9809009', '0704.0822']

In [18]:
print ('Recommendations ')
print (res_articles.loc[final_recommended_articles_id]['title'])

Recommendations 
id
0704.0921           Z^0 \to 2\gamma and the Twisted Coproduct of t...
0704.0640           Skew-Hadamard matrices of orders 188 and 388 e...
0704.0824                         On the (3,N) Maurer-Cartan equation
chao-dyn/9809009    Correspondence between classical dynamics and ...
0704.0822           Braided quantum field theories and their symme...
Name: title, dtype: object
